## You could execute TensorFlow operations imperatively,  directly from Python?
    ##  Eager Execution
    ## "A NumPy-like library for numerical computation with support for GPU acceleration
    ## and automatic differentiation, and a flexible platform
    ## for machine learning research and experimentation."

In [0]:
import tensorflow.contrib.eager as tfe



In [0]:
import tensorflow as tf
tfe.enable_eager_execution()

# 이렇게 작성 안해도 된다.
i = tf.constant(0)

while i < 1000 :
  i = tf.add(i,1)
  
  if i % 100 == 0 :
  
    print("i could do this %d" % i)

You no longer need to worry about ...

placeholders

sessions


control dependencies

"lazy loading"

{name, variable, op} scopes


In [3]:
x=[[2.]] # # No need for placeholders!
m = tf.matmul(x,x) ## # No sessions!
print(m)


tf.Tensor([[4.]], shape=(1, 1), dtype=float32)


In [4]:
x = tf.random_uniform([2,2])
for i in range(x.shape[0]):
    for j in range(x.shape[1]) :
        print(x[i,j])
    

tf.Tensor(0.25519526, shape=(), dtype=float32)
tf.Tensor(0.69116926, shape=(), dtype=float32)
tf.Tensor(0.85363877, shape=(), dtype=float32)
tf.Tensor(0.7082472, shape=(), dtype=float32)


## Tensors Act Like NumPy Arrays

In [8]:
import numpy as np
x = tf.constant([1.0, 2.0, 3.0])


# Tensors are backed by NumPy arrays
assert type(x.numpy()) == np.ndarray
squared = np.square(x) 
for i in x:
  print(i)


tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(3.0, shape=(), dtype=float32)


# Gradients



1.   Automatic differentiation is built into eager execution

Under the hood ...

Operations are recorded on a tape

The tape is played back to compute gradients

This is reverse-mode differentiation (backpropagation).





In [11]:
def square(x):
  return x ** 2

grad = tfe.gradients_function(square)  # Differentiate w.r.t. input of square 

print(square(3.))  # tf.Tensor(9., shape=(), dtype=float32)

print(grad(3.))      # [tf.Tensor(6., shape=(), dtype=float32))]

[<tf.Tensor: id=111, shape=(), dtype=float32, numpy=6.0>]


In [19]:
x = tfe.Variable(2.0) # Use tfe.Variable when eager execution is enabled.
def loss(y):
  return (y - x ** 2) ** 2

grad = tfe.implicit_gradients(loss) # Differentiate w.r.t. variables used to compute loss

print(loss(7.).numpy())  # tf.Tensor(9., shape=(), dtype=float32)
print(grad(7.))  # [(<tf.Tensor: -24.0, shape=(), dtype=float32>, 
                  #   <tf.Variable 'Variable:0' shape=()                
                 #     dtype=float32, numpy=2.0>)]


9.0
[(<tf.Tensor: id=7645, shape=(), dtype=float32, numpy=-24.0>, <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0>)]


# APIs for computing gradients work even when eager execution is not enabled


```
tfe.gradients_function()
tfe.value_and_gradients_function()
tfe.implicit_gradients()
tfe.implicit_value_and_gradients()
```











In [14]:
class Model(object):
  def __init__(self):
    self.W = tfe.Variable(5., name='weight')
    self.B = tfe.Variable(10., name='bias')

  def predict(self, inputs):
    return inputs * self.W + self.B


# The loss function to be optimized
def loss(model, inputs, targets):
  error = model.predict(inputs) - targets
  return tf.reduce_mean(tf.square(error))

# A toy dataset of points around 3 * x + 2
NUM_EXAMPLES = 1000
training_inputs = tf.random_normal([NUM_EXAMPLES])
noise = tf.random_normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise

# Define:
# 1. A model
# 2. Derivatives of a loss function with respect to model parameters
# 3. A strategy for updating the variables based on the derivatives
model = Model()
grad = tfe.implicit_gradients(loss)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

# The training loop
print("Initial loss: %f" %
      loss(model, training_inputs, training_outputs).numpy())
for i in range(201):
  optimizer.apply_gradients(grad(model, training_inputs, training_outputs))
  if i % 20 == 0:
    print("Loss at step %d: %f" %
          (i, loss(model, training_inputs, training_outputs).numpy()))
print("Final loss: %f" % loss(model, training_inputs, training_outputs).numpy())
print("W, B = %s, %s" % (model.W.numpy(), model.B.numpy()))

Initial loss: 69.222786
Loss at step 0: 66.535606
Loss at step 20: 30.357307
Loss at step 40: 14.179811
Loss at step 60: 6.945702
Loss at step 80: 3.710745
Loss at step 100: 2.264105
Loss at step 120: 1.617165
Loss at step 140: 1.327848
Loss at step 160: 1.198459
Loss at step 180: 1.140593
Loss at step 200: 1.114713
Final loss: 1.114713
W, B = 3.020185, 2.1190693
